In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/nyc-taxi-trip-duration/train.zip")
test = pd.read_csv("/kaggle/input/nyc-taxi-trip-duration/test.zip")
weather = pd.read_csv("/kaggle/input/new-york-weather/centralParkWeather.csv")

In [ ]:
train["pickup_datetime"] = pd.to_datetime(train["pickup_datetime"])
test["pickup_datetime"] = pd.to_datetime(test["pickup_datetime"])

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# tranforming the character to numeric values 
le = LabelEncoder()
le.fit(train['store_and_fwd_flag'])
train['store_and_fwd_flag'] = le.transform(train['store_and_fwd_flag'])
test['store_and_fwd_flag'] = le.transform(test['store_and_fwd_flag'])

In [ ]:
train.head()

In [ ]:
weather.head()

In [ ]:
# extract the pickup_datatime infos and store them in 
# independent variables 
train['year'] = train['pickup_datetime'].dt.year
train['month'] = train['pickup_datetime'].dt.month
train['day'] = train['pickup_datetime'].dt.day
train['weekday'] = train['pickup_datetime'].dt.weekday
train['hour'] = train['pickup_datetime'].dt.hour
train['minute'] = train['pickup_datetime'].dt.minute

test['month'] = test['pickup_datetime'].dt.month
test['day'] = test['pickup_datetime'].dt.day
test['weekday'] = test['pickup_datetime'].dt.weekday
test['hour'] = test['pickup_datetime'].dt.hour
test['minute'] = test['pickup_datetime'].dt.minute

In [ ]:
weather["DATE"] = pd.to_datetime(weather["DATE"])

In [ ]:
weather['month'] = weather['DATE'].dt.month
weather['day'] = weather['DATE'].dt.day
weather['year'] = weather['DATE'].dt.year

In [ ]:
weather.head()

In [ ]:
# only taking data of 2016
weather_2016 = weather[weather["year"]== 2016]

In [ ]:
weather_2016.drop(["NAME", "DATE", "STATION"],axis = 1, inplace = True)

In [ ]:
weather_2016.head()

In [ ]:
train.head()

In [ ]:
train["year"].unique()

In [ ]:
train["weekday"].value_counts()

In [ ]:
train["is_weekend"] = train["weekday"] > 4
test["is_weekend"] = test["weekday"] > 4

In [ ]:
train.head()

In [ ]:
from matplotlib import pyplot

In [ ]:
import math

In [ ]:
EARTH_RADIUS = 6378.137  ## km
def haversine(xy1, xy2):
    return 2 * EARTH_RADIUS * np.arcsin(np.sqrt(
        np.sin((xy2[:,0]-xy1[:,0])/2)**2 +
        np.cos(xy1[:,0])*np.cos(xy2[:,0])*np.sin((xy2[:,1]-xy2[:,1])/2)
    ))
train["distance"] = haversine(
    np.radians(train[["pickup_longitude", "pickup_latitude"]].values),
    np.radians(train[["dropoff_longitude", "dropoff_latitude"]].values))
test["distance"] = haversine(
    np.radians(test[["pickup_longitude", "pickup_latitude"]].values),
    np.radians(test[["dropoff_longitude", "dropoff_latitude"]].values))

pyplot.hist(np.log(train["distance"]+1e-5), bins=50)

In [ ]:
train.head()

In [ ]:
X = train.drop(["trip_duration", "id", "vendor_id", "pickup_datetime", "dropoff_datetime"], axis=1)
y = train["trip_duration"]
X_test = test.drop(["id", "vendor_id", "pickup_datetime"], axis=1)

In [ ]:
np.setdiff1d(df_train.columns, df_test.columns)

In [ ]:
features = [column for column in train.columns if column not in ['avg_speed_h', 'dropoff_datetime', 'log_trip_duration',\
                                                  'trip_duration', 'id', 'pickup_datetime', 'dropoff_datetime',\
                                                  'distance_haversine',\
                                                 'trip_duration', 'log_trip_duration', 'trip_duration','pickup_date']]

In [ ]:
train_filter = train.dropna()
X = train_filter[features].values
y = np.log(train_filter['trip_duration'].values + 1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=7)
X_test = df_test[features].values

In [ ]:
#Split the data into training, test, and valdiation sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2019)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'max_depth': np.arange(3, 10)}
tree = GridSearchCV(DecisionTreeRegressor(), param_grid)

In [ ]:
tree.fit(X_train, y_train)

In [ ]:
y_pred = tree.predict(X_val)

In [ ]:
# submission
ytest = tree.predict(X_test)
test['trip_duration'] = np.exp(ytest) - 1
test[['id', 'trip_duration']].to_csv('dt_submission.csv.gz', index=False, compression='gzip')